In [0]:
from delta.tables import DeltaTable

In [0]:
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# #Version 1
# data = [("James","","Smith","36636","M",3000),
#     ("Michael","Rose","","40288","M",4000),
#     ("Robert","","Williams","42114","M",4000),
#     ("Maria","Anne","Jones","39192","F",4000),
#     ("Jen","Mary","Brown","","F",-1)
#   ]

# schema = StructType([ \
#     StructField("firstname",StringType(),True), \
#     StructField("middlename",StringType(),True), \
#     StructField("lastname",StringType(),True), \
#     StructField("id", StringType(), True), \
#     StructField("gender", StringType(), True), \
#     StructField("salary", IntegerType(), True) \
#   ])
 
# df = spark.createDataFrame(data=data,schema=schema)
# df.write.mode("overwrite").save("/mnt/delta/employee")

# #Version 2
# data = [("Sacha","","Smith","36636","M",3000),
#     ("Dimitri","Rose","","40288","M",4000),
#     ("Raskolnikov","","Williams","42114","M",4000),
#     ("Macha","Anne","Jones","39192","F",4000),
#     ("Anton","Mary","Brown","","F",-1)
#   ]

# schema = StructType([ \
#     StructField("firstname",StringType(),True), \
#     StructField("middlename",StringType(),True), \
#     StructField("lastname",StringType(),True), \
#     StructField("id", StringType(), True), \
#     StructField("gender", StringType(), True), \
#     StructField("salary", IntegerType(), True) \
#   ])
 
# df = spark.createDataFrame(data=data,schema=schema)
# df.write.mode("overwrite").save("/mnt/delta/employee")

In [0]:
employees = spark.read.load("/mnt/delta/employee", format="delta")
employees.show()

+-----------+----------+--------+-----+------+------+
 firstname|middlename|lastname| id|gender|salary|
+-----------+----------+--------+-----+------+------+
Raskolnikov| |Williams|42114| M| 4000|
 Macha| Anne| Jones|39192| F| 4000|
 Dimitri| Rose| |40288| M| 4000|
 Sacha| | Smith|36636| M| 3000|
 Anton| Mary| Brown| | F| -1|
+-----------+----------+--------+-----+------+------+

In [0]:
employees_for_history = DeltaTable.forPath(spark, "/mnt/delta/employee")
display(employees_for_history.history())

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-05-26T07:22:50.000+0000,808098371030088,nastasia.saby@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(142234619150852),0526-055223-sine958,0,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 8552, numOutputRows -> 5)",null
0,2021-05-26T07:22:44.000+0000,808098371030088,nastasia.saby@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(142234619150852),0526-055223-sine958,null,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 8487, numOutputRows -> 5)",null


In [0]:
first_version_of_employees = spark.read.option("versionAsOf", 0).format("delta").load("/mnt/delta/employee")
display(first_version_of_employees)

firstname,middlename,lastname,id,gender,salary
Maria,Anne,Jones,39192,F,4000
Robert,,Williams,42114,M,4000
Michael,Rose,,40288,M,4000
James,,Smith,36636,M,3000
Jen,Mary,Brown,,F,-1


In [0]:
second_version_of_employees = spark.read.format("delta").load("/mnt/delta/employee")
display(second_version_of_employees)

firstname,middlename,lastname,id,gender,salary
Raskolnikov,,Williams,42114,M,4000
Macha,Anne,Jones,39192,F,4000
Dimitri,Rose,,40288,M,4000
Sacha,,Smith,36636,M,3000
Anton,Mary,Brown,,F,-1


In [0]:
#Be aware that this timestamp could not suit your data
#Choose the timestamp regarding the history of your table
first_version_of_employees = spark.read.option("timestampAsOf", "2021-05-03T15:06:07.000+0000").format("delta").load("/mnt/delta/employee")
display(first_version_of_employees)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-142234619150857> in <module> 
 ----> 1 first_version_of_employees = spark . read . option ( "timestampAsOf" , "2021-05-03T15:06:07.000+0000" ) . format ( "delta" ) . load ( "/mnt/delta/employee" ) 
 2 display ( first_version_of_employees ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 202 self . options ( ** options ) 
 203 if isinstance ( path , str ) : 
 --> 204 return self . _df ( self . _jreader . load ( path ) ) 
 205 elif path is not None : 
 206 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o831.load.
: com.databricks.sql.transaction.tahoe.DeltaErrors$TimestampEarlierThanCommitRetentionException: The provided timestamp (2021-05-03 15:06:07.0) is before the earliest version available to this
table (2021-05-26 07:22:44.0). Please use a timestamp after 2021-05-26 07:22:44.
 
	at com.databricks.sql.transaction.tahoe.DeltaHistoryManager.getActiveCommitAtTime(DeltaHistoryManager.scala:139)
	at com.databricks.sql.transaction.tahoe.DeltaTableUtils$.resolveTimeTravelVersion(DeltaTable.scala:425)
	at com.databricks.sql.transaction.tahoe.catalog.DeltaTableV2.$anonfun$snapshot$1(DeltaTableV2.scala:98)
	at scala.Option.map(Option.scala:230)
	at com.databricks.sql.transaction.tahoe.catalog.DeltaTableV2.snapshot$lzycompute(DeltaTableV2.scala:96)
	at com.databricks.sql.transaction.tahoe.catalog.DeltaTableV2.snapshot(DeltaTableV2.scala:95)
	at com.databricks.sql.transaction.tahoe.catalog.DeltaTableV2.toBaseRelation(DeltaTableV2.scala:157)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:209)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:390)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:424)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$1(DataFrameReader.scala:389)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:358)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:278)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)